In [1]:
import os

# Set the working directory to your project's root directory on D drive
os.chdir("D:\Pycharm_projects")



In [3]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from IPython.display import display as Markdown
from tqdm.autonotebook import tqdm as notebook_tqdm


In [4]:
import os
import PyPDF2

# Verify Python environment and paths
print("Current Working Directory:", os.getcwd())

# Path to your PDF file
pdf_path = "D:\\Pycharm_projects\\mono.pdf"  # Ensure the file path is correct

# Initialize data list to store text from each page
data = []

with open(pdf_path, "rb") as file:
    reader = PyPDF2.PdfReader(file)
    text = ""

    # Iterate through all pages and extract text
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        page_text = page.extract_text()

        # Store each page's number and extracted text in the data variable
        data.append({
            'page_number': page_num + 1,  # Page numbers are 1-indexed
            'page_content': page_text
        })

    # Print the first 500 characters of the extracted text from the first page
    print("Text extracted successfully from the first page:", data[0]['page_content'][:500])

# Optional: Print the data variable (can be large, depending on the PDF content)
# print(data)


Current Working Directory: D:\Pycharm_projects
Text extracted successfully from the first page: MONOPOLY 
Property Trading Game from Parker Brothers" 
AGES 8+ 
2 to 8 Players 
Contents: Gameboard, 3 dice, tokens, 32 houses, I2 hotels, Chance 
and Community Chest cards, Title Deed cards, play money and a Banker's tray. 
Now there's a faster way to play MONOPOLY. Choose to play by 
the classic rules for buying, renting and selling properties or use the 
Speed Die to get into the action faster. If you've never played the classic 
MONOPOLY game, refer to the Classic Rules beginning on the next


In [5]:
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# Assuming 'data' has already been populated with 'page_content' as explained before

# Convert the data list of dictionaries into a list of Document objects
documents = [Document(page_content=page['page_content']) for page in data]

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=200)

# Split the documents into chunks
chunks = text_splitter.split_documents(documents)

# Print the first chunk as an example
print("First chunk:", chunks[0])


First chunk: page_content='MONOPOLY 
Property Trading Game from Parker Brothers" 
AGES 8+ 
2 to 8 Players 
Contents: Gameboard, 3 dice, tokens, 32 houses, I2 hotels, Chance 
and Community Chest cards, Title Deed cards, play money and a Banker's tray. 
Now there's a faster way to play MONOPOLY. Choose to play by 
the classic rules for buying, renting and selling properties or use the 
Speed Die to get into the action faster. If you've never played the classic 
MONOPOLY game, refer to the Classic Rules beginning on the next page. 
If you already know how to play and want to use the Speed Die, just 
read the section below for the additional Speed Die rules. 
SPEED DIE RULES 
Learnins how to Play with the S~eed Die IS as 
/ 
fast as playing with i't. 
1. When starting the game, hand out an extra $1,000 to each player 
(two $5005 should work). The game moves fast and you'll need 
the extra cash to buy and build. 
2. Do not use the Speed Die until you've landed on or passed over 
GO for the 

In [7]:
!ollama list

NAME                       ID              SIZE      MODIFIED     
mistral:latest             f974a74358d6    4.1 GB    23 hours ago    
nomic-embed-text:latest    0a109f422b47    274 MB    25 hours ago    
llama3.2:latest            a80c4f17acd5    2.0 GB    26 hours ago    


In [8]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

# Ensure 'chunks' is correctly populated with Documents (not strings or dicts)
vector_db = Chroma.from_documents(
    documents=chunks,  # Ensure 'chunks' is valid
    embedding=OllamaEmbeddings(model="nomic-embed-text"),  # Check model validity
    collection_name="local-rag"
)


In [11]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [12]:
local_model = "mistral:latest"
llm = ChatOllama(model=local_model)

In [13]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [14]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(),
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [15]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [16]:
chain.invoke("what is banker")

ResponseError: llama runner process has terminated: error loading model: unable to allocate backend buffer

In [ ]:
vector_db.delete_collection()

In [2]:
import os
import PyPDF2
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq




os.chdir("D:\Pycharm_projects")


pdf_path = "D:\\Pycharm_projects\\mono.pdf"
data = []

with open(pdf_path, "rb") as file:
    reader = PyPDF2.PdfReader(file)


    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        page_text = page.extract_text()

        data.append({
            'page_number': page_num + 1,
            'page_content': page_text
        })


documents = [Document(page_content=page['page_content']) for page in data]


text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)


os.environ['GROQ_API_KEY'] = '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'


embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)


vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    collection_name="huggingface-groq-rag"
)


llm = ChatGroq(
    temperature=0,
    model_name="mixtral-8x7b-32768",
    groq_api_key=os.environ['GROQ_API_KEY']
)
from langchain_core.prompts import PromptTemplate

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI assistant generating alternative query perspectives.
    Generate 5 different versions of the given question to improve document retrieval:
    Original question: {question}"""
)


retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(),
    llm,
    prompt=QUERY_PROMPT
)


template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)


chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


result = chain.invoke("summarise it")
print(result)


The documents provide rules for the game Monopoly. Each player starts with $1,500 and the Bank holds all remaining money and properties. The Bank pays salaries, sells properties, and collects taxes, fines, and loans. Players move around the board based on the roll of the dice, and may buy properties, pay rent, or perform other actions based on the space they land on. Players can buy houses and hotels to increase rent collection. Properties can be sold to other players or mortgaged through the Bank. There are special spaces on the board such as "Free Parking," "Chance," and "Community Chest" that allow players to draw cards and perform additional actions. The objective of the game is to become the wealthiest player through buying, renting, and selling properties.
